本文介绍了Stable Diffusion是一个文本到图像的潜在扩散模型的入门介绍及使用教程。

Stable Diffusion是一个文本到图像的潜在扩散模型，而其最火的产品由CompVis、Stability AI和LAION的研究人员和工程师创建。它使用来自LAION-5B数据库子集的512x512图像进行训练。使用这个模型，可以生成包括人脸在内的任何图像，因为有开源的预训练模型，所以我们也可以在自己的机器上运行它。

https://blog.csdn.net/lgzlgz3102/article/details/128527028

首先介绍什么是Stable Diffusion，并讨论它的主要组成部分。

## Stable Diffusion model

Stable Diffusion（稳定扩散）是一种基于 Langevin Dynamics 的模型，用于对概率分布进行建模。

其中 Langevin Dynamics 是一种基于随机梯度下降算法的优化方法，它将随机噪声加入到梯度下降过程中，以帮助算法更好地跳出局部极值。

在 Stable Diffusion 模型中，Langevin Dynamics 被用于进行概率分布的扩散和混合。它通过将噪声逐渐加深，将初始分布扩散到更广泛的空间，以便更好地捕获数据分布的特征。

因此，Stable Diffusion 可以用于生成高质量的样本，并且可以很好地应用于图像生成、文本生成等领域。

### Diffusion Models

扩散模型 (Diffusion Models) 是一种新型的、先进的生成模型，可以生成各种高分辨率图像。在 OpenAI, Nvidia 和 Google 成功地训练了大规模的模型后，扩散模型已经吸引了很多人的注意。基于扩散模型的架构有 GLIDE, DALLE-2, Imagen 和 完全开源的 Stable Diffusion。

工作原理是通过一系列的 T 步骤将高斯噪声逐渐添加到原始图像中，这个过程被称为扩散。




基于生成对抗网络 GANs 的 AI 生成图像往年在互联网不温不热，但就在最近这几个月， Open AI 于 2022 年 4 月初发布的 DALL-E 2 (基于 GPT Model) ，以及 stability.ai 于 2022 年 8 月底发布的 Stable Diffusion (基于 Diffusion Models) ，其生成的照片、画作的效果让人乍舌，随即引发了一股新的互联网 AI 创作热潮。

引发这一系列热潮的便是 Novel AI 于 2022 年 10 月初发布的，能够画各种精致二次元风格图片的 NovelAI Diffusion (基于 Stable Diffusion) 一下子把 AI 绘画推向风口浪尖。无数乐子人蜂拥而至，甚至有黑客把 Novel AI 的官网源码和模型全部扒了下来。 AI 创作数量极快，乍一看都很精致，而这些 “AI Based” 作品正在以一种极快的速度挤压创作者的空间，随即引发的便是关于伦理道德和法律的一系列疑问。

但是本文暂不讨论伦理道德等方面的问题（也许之后会写一篇文章讨论），仅仅先从技术角度和数学原理上简要介绍效果出众、“秒杀” GANs 且改变了人们对原本 AI 绘画认知的 Diffusion Models 的数学原理。

https://shao.fun/blog/w/how-diffusion-models-work.html#%E8%AF%91%E8%80%85%E5%89%8D%E8%A8%80

我们将重点讨论其中最突出的一个，即由 Sohl-Dickstein et al 和 Ho. et al 2020 提出的去噪扩散概率模型 (DDPM, denoising diffusion probabilistic model) 。有意思的是，DDPM的数学框架其实在ICML2015的论文[《Deep Unsupervised Learning using Nonequilibrium Thermodynamics》](https://arxiv.org/abs/1503.03585)就已经完成了

其它各种方法将不会具体讨论，如 Stable Diffusion 和 score-based models 。

### 去噪扩散概率模型 (DDPM, denoising diffusion probabilistic model) 





### 前向扩散

俗话：“破坏容易建设难”，建楼你不会，拆楼你总会了吧？我们考虑将高楼大厦一步步地拆为砖瓦水泥的过程：设$x_0$
为建好的高楼大厦（数据样本），$x_T$
为拆好的砖瓦水泥（随机噪声），假设“拆楼”需要 T
步，整个过程可以表示为

$$ x = x_0→x_1→x_2→⋯→x_{T−1}→x_T=z $$



类似地，DDPM的前向扩散过程也是如此。

先将噪声形成的过程设为一个T步的马尔科夫链，该假设是相当自然的，因为在很多现实问题中，我们只关心当前状态的概率，而不需要考虑更早时刻的状态，将其记为

$$
\{x_0,x_1,...,x_T\}
$$

其中$x_0$是从真实数据分布$q(x)$中采样，即 $x \sim q(x)$

马尔可夫链的性质之一：无后效性，即当前状态的概率只依赖于前一时刻的状态，而不依赖于更早的状态，如果我们用 $X_t$ 表示时刻 $t$ 的状态（比如一个随机变量），那么满足马尔可夫性的过程可以用下面的条件概率分布来表示：

$$
P(X_{t+1}=x_{t+1} \mid X_t=x_t, X_{t-1}=x_{t-1}, \cdots, X_0=x_0) = P(X_{t+1}=x_{t+1} \mid X_t=x_t)
$$



![前向扩散过程 图片修改自 [Ho et al. 2020](https://arxiv.org/abs/2006.11239)](https://shao.fun/assets/forward-diffusion.8f41da07.png)

我们通过添加噪声来定义一个前向扩散过程，具体来说，在马尔可夫链的每一步，我们向$x_{t-1}$添加方差为$\beta_t$的高斯噪音，来产生一个新的潜在变量$x_{t}$，根据概率论的乘法法则，也就是乘上一个条件概率分布$\mathrm{q}\left(x_t \mid x_{t-1}\right) = \mathcal{N}\left(x_t ; \sqrt{1-\beta_t} x_{t-1}, \beta_T I\right)$，这里的$\beta_{t}$是控制扩散速率的一个参数，也称为diffusion schedule，每个时刻t都有自己的$\beta$）。

由于我们处于多维情况下，$\mathbf{I}$是单位矩阵，表明每个维度有相同的标准偏差 $\beta_t$ 。注意到， $q\left(\mathbf{x}_t \mid \mathbf{x}_{t-1}\right)$ 是一个正态分布，其均值是 $\boldsymbol{\mu}_t=\sqrt{1-\beta_t} \mathbf{x}_{t-1}$ ，方差为 $\boldsymbol{\Sigma}_t=\beta_t \mathbf{I}$ ，其中 $\boldsymbol{\Sigma}$ 是一个对角矩阵的方差 (这里就是 $\beta_t$ ) 。

因此，我们可以自 $\mathbf{x}_0$ 到 $\mathbf{x}_t$ 以一种可操作的方式来近似输入。在数学上，这种后验概率定义如下:

$$
q\left(\mathbf{x}_{1: T} \mid \mathbf{x}_0\right)=\prod_{t=1}^T q\left(\mathbf{x}_t \mid \mathbf{x}_{t-1}\right)
$$

其中， $q\left(\mathbf{x}_{1: T}\right)$ 意味着我们从时间 1 到 $T$ 重复应用 $q$ 。
到目前为止，看起来还不错? 并不!对于时间步长 $t=500<T$ 我们需要为了采样 $\mathbf{x}_t$ 应用 $q$ 函数 500 次。难道就没有更好的方式吗?

[重参数化技巧 (Reparametrization Trick)](https://theaisummer.com/latent-variable-models/#reparameterization-trick)对此提供了一个魔法般的补救办法。


$p_\theta\left(x_{t-1} \mid x_t\right)$都是高斯分布$\mathcal{N}\left(x_{t-1} ; \mu_\theta\left(x_t, t\right), \Sigma_\theta\left(x_t, t\right)\right)$

条件概率分布$q\mathrm{q}\left(x_t \mid x_{t-1}\right)$

但它的均值、方差需要训练模型学习拟合。由于在训练过程中，我们只有目标数据分布和高斯噪声，无法得知$x_T$相关信息，因此无法直接用于训练。为解决该问题，DDPM提出了$\mathrm{q}\left(x_t \mid x_0\right)$



#### 针对图像应用的过程

Diffusion models是一种基于偏微分方程的图像去噪算法，其主要思想是通过对噪声信号进行扩散处理，逐渐降低噪声的强度，从而实现去噪的目的。

**基本原理**：把输入图像$x_0$并通过一系列的$T$步骤，逐渐向其添加高斯噪声，我们将此称为正向过程。值得注意的是，正向过程与神经网络的正向传播无关，但是正向传播对于为我们的神经网络生成目标（应用  噪声步骤后的图像）是有必要的。之后，*神经网络*被训练为通过逆转噪声过程来恢复原始数据。通过*对反向过程进行建模*，我们可以生成新的数据。这就是所谓的反向扩散过程，或者说是生成式模型的采样过程。

下面是扩散模型的详细过程：

1. 数据准备：首先需要准备噪声图像数据，可以使用已有的图像库或自行生成噪声图像。
1. 用扩散处理来加噪：对噪声图像进行扩散处理，使得噪声逐渐弱化。扩散的过程可以使用随机游走或扩散算子来实现。随机游走是一种随机过程，用于模拟随机漫步，可以模拟噪声在图像中的扩散过程。扩散算子是一种线性算子，用于模拟扩散过程。在每个时间步骤，都会对噪声图像进行一次扩散处理。
1. 用神经网络建模去噪过程：通过梯度下降等优化算法，对扩散后的噪声图像进行去噪处理。去噪的目标是最小化噪声图像与原始图像之间的误差，通常使用平均平方误差（MSE）或结构相似性（SSIM）等指标来评估去噪效果。在每个时间步骤，都会对去噪后的图像进行一次优化处理。
1. 迭代：重复进行噪声扩散和去噪处理，直到达到预设的迭代次数或去噪效果满足要求为止。每次迭代都会对扩散后的噪声图像进行一次去噪处理，逐步恢复出原始图像。
1. 输出结果：最终输出去噪后的图像结果，以及迭代过程中的一些参数信息，如每次迭代的去噪效果、时间消耗等。



### 隐式扩散模型（Implicit Diffusion Model，IDM）

隐式扩散模型（Implicit Diffusion Model，IDM）是一种用于建模扩散过程的概率模型，常用于生成式模型（Generative Model）和图像处理任务中。

在扩散过程中，噪声信号会逐渐扩散并被混合，从而形成最终的数据。IDM的核心思想是将扩散过程建模为一系列概率密度函数的乘积，每个概率密度函数表示当前时间点下的噪声分布，而它的乘积则表示整个扩散过程的概率密度函数。

与其他扩散模型不同的是，IDM并不显式地定义每个时间点下的概率密度函数，而是通过一个神经网络来隐式地学习这些函数的表达式。具体来说，IDM使用一个神经网络作为条件密度函数的近似，输入是当前时间点的噪声信号，输出是噪声信号在下一个时间点的概率密度函数的参数。

通过不断地重复应用条件密度函数，可以得到整个扩散过程的概率密度函数。在训练时，IDM通过最大似然估计或变分推断等方法来优化神经网络的参数，以最大化模型对数据的拟合度。

IDM的一个优点是可以处理高维数据，同时在训练过程中不需要计算扩散过程的精确概率密度函数。此外，IDM的生成速度相对较快，并且可以轻松地将其与其他生成式模型进行集成。

#### 数学公式

隐式扩散模型（IDM）的核心公式可以表示为：

$$
p_t(\mathbf{x}) = \int p_{t-1}(\mathbf{y}) p(\mathbf{x} \mid \mathbf{y}, t) d\mathbf{y}
$$


其中，$p_t(\mathbf{x})$ 表示在时间 $t$ 时，噪声信号的概率密度函数；$p_{t-1}(\mathbf{y})$ 表示在时间 $t-1$ 时，噪声信号的概率密度函数；$p(\mathbf{x} \mid \mathbf{y}, t)$ 表示在给定 $\mathbf{y}$ 的条件下，噪声信号在时间 $t$ 时的概率密度函数。

通常，$p(\mathbf{x} \mid \mathbf{y}, t)$ 由一个神经网络表示，可以写成：
​
其中，$\mathcal{N}(\cdot)$ 表示高斯分布，$\boldsymbol{\mu}_t(\mathbf{y})$ 和 $\boldsymbol{\Sigma}_t(\mathbf{y})$ 分别表示给定 $\mathbf{y}$ 后的均值和协方差矩阵。这里的 $\boldsymbol{\mu}_t(\mathbf{y})$ 和 $\boldsymbol{\Sigma}_t(\mathbf{y})$ 也可以通过一个神经网络来表示。

$$
p(\mathbf{x} \mid \mathbf{y}, t) = \mathcal{N}(\mathbf{x} ; \boldsymbol{\mu}_t(\mathbf{y}), \boldsymbol{\Sigma}_t(\mathbf{y}))
$$

在训练过程中，通常采用最大似然估计（Maximum Likelihood Estimation，MLE）的方法，即最大化训练数据的联合概率密度函数。具体来说，可以将联合概率密度函数表示为各个时间点概率密度函数的乘积，然后使用随机梯度下降等优化方法来最大化该函数。

在隐式扩散模型（IDM）中，噪声信号的条件概率密度函数 $p(\mathbf{x} \mid \mathbf{y}, t)$ 是由一个神经网络表示的。具体来说，该神经网络的输入是 $\mathbf{y}$ 和 $t$，输出是均值向量 $\boldsymbol{\mu}_t(\mathbf{y})$ 和协方差矩阵 $\boldsymbol{\Sigma}_t(\mathbf{y})$，它们用于定义高斯分布 $p(\mathbf{x} \mid \mathbf{y}, t)$。

通常，这个神经网络可以采用卷积神经网络（Convolutional Neural Network，CNN）或者全连接神经网络（Fully Connected Neural Network，FCN）来实现。在训练过程中，可以使用最大似然估计（Maximum Likelihood Estimation，MLE）来训练神经网络，即最大化训练数据的联合概率密度函数。具体来说，可以将联合概率密度函数表示为各个时间点概率密度函数的乘积，然后使用随机梯度下降等优化方法来最大化该函数。

需要注意的是，由于 IDM 使用的是隐式方法，因此在训练过程中需要对时间步长进行控制，以保证数值稳定性和精度。此外，IDM 也可以通过多步扩散来提高模型的表达能力和预测精度。在这种情况下，神经网络需要输出多个均值向量和协方差矩阵，用于定义多个高斯分布。

#### DDPM跟Stable Diffusion关联大吗？

DDPM和Stable Diffusion之间存在一定的关联，因为它们都是基于扩散过程的生成式模型，且都使用了隐式扩散模型（IDM）来对扩散过程进行建模。

Stable Diffusion是一种改进的扩散模型，它在标准的扩散过程中引入了一个额外的稳定过程（Stable Process），从而可以更好地处理长尾分布的数据。Stable Diffusion的关键在于建立一个合适的稳定过程模型，用于描述噪声在扩散过程中的长尾分布特性。

DDPM同样使用了IDM来建模扩散过程，但它采用的是一个更为通用的方法，即使用神经网络来近似条件密度函数。与Stable Diffusion不同的是，DDPM并没有专门考虑长尾分布的特性，因此在处理长尾分布的数据时可能会出现一些限制。

然而，最近的研究表明，DDPM也可以通过一些简单的修改来支持长尾分布的数据。例如，可以在IDM中引入一些额外的参数来调整噪声分布的形态，从而更好地适应不同的数据分布。

Stable Diffusion 算法上来自 CompVis 和 Runway 团队于 2021 年 12 月提出的 “潜在扩散模型”（LDM / Latent Diffusion Model），这个模型又是基于 2015 年提出的扩散模型（DM / Diffusion Model）。参考论文中介绍算法核心逻辑的插图，Stable Diffusion 的数据会在像素空间（**Pixel Space**）、潜在空间（**Latent Space**）、条件（**Conditioning**）三部分之间流转，其算法逻辑大概分这几步（可以按 ↩️ 顺序对照下图）：

1. 图像编码器将图像从像素空间（**Pixel Space**）压缩到更小维度的潜在空间（**Latent Space**），捕捉图像更本质的信息；
对潜在空间中的图片添加噪声，进行扩散过程（**Diffusion Process**）；
1. 通过 CLIP 文本编码器将输入的描述语转换为去噪过程的条件（**Conditioning**）；
1. 基于一些条件对图像进行去噪（Denoising）以获得生成图片的潜在表示，去噪步骤可以灵活地以文本、图像和其他形式为条件（以文本为条件即 text2img、以图像为条件即 img2img）；
1. 图像解码器通过将图像从潜在空间转换回像素空间来生成最终图像。

潜在扩散模型对算力、显卡性能的要求显著降低。这也就是为什么 Stable Diffusion 能在消费级显卡上运行，从而让 AI 绘画 “飞入寻常百姓家”。

猜测：之所以这个基于 Latent Diffusion 的模型叫 Stable Diffusion，可能一方面表示这个模型效果很稳定（Stable），另一方面是致敬一下（算力 & 数据上的）金主爸爸 Stability.ai。

https://www.nolibox.com/creator_articles/principle_of_stablediffusion.html



### Stable Diffusion与普通Diffusion模型的两点区别

为了将这些U型网扩展到高分辨率的图像中，在计算上是非常昂贵的。这时有两种将扩散模型扩展到高分辨率的方法：级联扩散模型和潜伏扩散模型。

#### 级联扩散模型

Ho等人在2021年引入了级联扩散模型，以努力产生高保真的图像。级联扩散模型包括一个由许多连续扩散模型组成的管道，生成分辨率越来越高的图像。每个模型通过连续地对图像进行上采样并增加更高分辨率的细节，生成一个比前一个质量更好的样本。为了生成一个图像，我们从每个扩散模型中依次取样。

![](https://www.datalearner.com/resources/blog_images/a3a33132-3c08-4c35-b009-04f685ed15c6.png)

为了获得级联架构的良好效果，对每个超级分辨率模型的输入进行强有力的数据增强是至关重要的。为什么呢？因为它可以减轻之前级联模型的复合误差，以及由于训练-测试不匹配造成的误差。

研究发现，高斯模糊是实现高保真度的一个关键转变。他们把这种技术称为调节增强。

扩散模型有一个主要的缺点就是去噪过程的时间和内存消耗都非常昂贵。这会使进程变慢，并消耗大量内存。主要原因是它们在像素空间中运行，特别是在生成高分辨率图像时。

#### 稳定的扩散（Stable diffusion）：潜在扩散模型

潜伏扩散模型 Latent diffusion通过在较低维度的潜空间（Latent Space)上应用扩散过程而不是使用实际的像素空间那种高维输入上，使用变分自编码器进行向上和向下取样，来减少内存和计算成本。所以Stable Diffusion引入了Latent diffusion的方式来解决这一问题计算代价昂贵的问题。

更详细地说， [Rombach et al.](https://arxiv.org/abs/2112.10752) 建议使用编码器网络将输入编码为潜伏表示，即$z_t=g(X_t)$  。这一决定背后的直觉是通过在低维空间处理输入来降低训练扩散模型的计算需求。之后，一个标准的扩散模型 (U-Net) 应用于生成新的数据，这些数据被一个解码器网络放大。




### 1、 Latent diffusion的主要组成部分

#### 自动编码器(VAE)

自动编码器(VAE)由两个主要部分组成:编码器和解码器。编码器将把图像转换成低维的潜在表示形式，该表示形式将作为下一个组件U_Net的输入。解码器将做相反的事情，它将把潜在的表示转换回图像。

在Latent diffusion训练过程中，利用编码器获得正向扩散过程中输入图像的潜表示(latent)。而在推理过程中，VAE解码器将把潜信号转换回图像。

#### U-Net

U-Net也包括编码器和解码器两部分，两者都由ResNet块组成。编码器将图像表示压缩为低分辨率图像，解码器将低分辨率解码回高分辨率图像。

为了防止U-Net在下采样时丢失重要信息，通常在编码器的下采样的ResNet和解码器的上采样ResNet之间添加了捷径的连接。

在Stable Diffusion的U-Net中添加了交叉注意层对文本嵌入的输出进行调节。交叉注意层被添加到U-Net的编码器和解码器ResNet块之间。

#### Text-Encoder

文本编码器将把输入文字提示转换为U-Net可以理解的嵌入空间，这是一个简单的基于transformer的编码器，它将标记序列映射到潜在文本嵌入序列。从这里可以看到使用良好的文字提示以获得更好的预期输出。

## Latent Diffusion的作用

### 为什么Latent Diffusion快速有效？

Latent Diffusion之所以快速有效，是因为它的U-Net是在低维空间上工作的。与像素空间扩散相比，这降低了内存和计算复杂度。例如，一个(3,512,512)的图像在潜在空间中会变成(3,64,64)，内存将会减少64倍。（可计算3*512*512/34*64*64）=64）

![潜伏的扩散模型](https://shao.fun/assets/stable-diffusion.0b168bae.png)

#### 潜在空间（Latent Space）

如何理解 “潜在空间” 呢？我们不需要用什么DNA，而是用身份证号码就能代表你自己，前 6 位代表地区、中间 8 位代表生日、后 4 位代表个人其他信息。放到空间上如图所示，这个空间就是「中国人的潜在空间」。

![中国人的潜在空间](https://localimage.nolibox.com/article_img_url/IsF0dkJW0okpvcx0PgtckNajnZb-boxcnmuNfOSUoHzoWxQQkAxhEdg.png?x-oss-process=image/resize,w_1500)

### 潜在扩散模型 (LDM) 的损失函数


如果一个典型的扩散模型 (DM) 的损失被表述为:

$$
L_{D M}=E_{x, t, \epsilon}\left[\left\|\epsilon-\epsilon_\theta\left(\mathbf{x}_t, t\right)\right\|^2\right]
$$

然后，给定编码器 $\mathcal{E}$ 和一个潜在表示 $z$，那么一个潜在扩散模型 (LDM) 的损失函数可以表示为：

$$
L_{D M}=E_{\xi(x), t, \epsilon}\left[\left\|\epsilon-\epsilon_\theta\left(\mathbf{z}_t, t\right)\right\|^2\right]
$$


## Stable Diffusion的推理过程

![Stable Diffusion的推理过程](https://img-blog.csdnimg.cn/img_convert/ff110cca10e7fe7078600561fe26671d.png)

### CLIP

CLIP 是一个使用自监督学习技术在大规模文本和图像数据上进行训练的模型，能够有效地捕捉文本和图像之间的关系。CLIP 的训练过程不需要任何监督信号，而是通过最大化图像和文本之间的余弦相似度来训练模型。

### 对比学习

对比学习（Contrastive Learning）是一种自监督学习（Self-supervised Learning）的方法。自监督学习是指利用无标签的数据进行学习的方法，它可以从大量的未标注数据中学习到有用的表示，这些表示可以用于许多下游任务，如分类、检索等。

在对比学习中，模型通过比较不同的样本之间的相似性来学习良好的特征表示。具体来说，模型将每个样本映射到一个低维向量空间中，然后通过比较向量之间的距离或相似度来学习有用的特征表示。在训练过程中，模型会通过最大化相似样本之间的相似性，最小化不同样本之间的相似性来优化模型参数，从而学习到有用的特征表示。

图片就是一系列像素点，文本就是一串字符，那AI如何完成图文匹配任务？

![](https://localimage.nolibox.com/article_img_url/IsF0dkJW0okpvcx0PgtckNajnZb-boxcnZ69awBYtL8xCirxaw0gMtg.png)

### CLIP


CLIP（Contrastive Language-Image Pre-training/对比式语言-文字预训练）是 OpenAI 发布的一种基于对比学习的自然语言处理（NLP）和计算机视觉（CV）的预训练模型,实现了图文的匹配。

![](https://localimage.nolibox.com/article_img_url/IsF0dkJW0okpvcx0PgtckNajnZb-boxcnOJzg5h6vnZ0aN38UbPKVVf.png)

CLIP 结合了图像和文本信息，可以同时理解图像和文本输入，并可以在许多 NLP 和 CV 任务上进行微调。

AI 在海量「文本-图片」数据上学习图片和文本的匹配。图中绿色方块是「图片潜在空间」的 N 张图片，紫色方块是「文本潜在空间」的 N 句描述语。AI 会努力将对应的 I1 与 T1 （蓝色方块）匹配，而不是 I2 与 T3 （灰色方块）匹配。

![](https://localimage.nolibox.com/article_img_url/IsF0dkJW0okpvcx0PgtckNajnZb-boxcnOJzg5h6vnZ0aN38UbPKVVf.png)

CLIP 的训练方法基于对比学习，其目标是将每个图像和文本输入嵌入到一个共同的向量空间中，并将相似的图像和文本输入映射到空间中的相邻点。这使得 CLIP 能够将具有相似语义的图像和文本对应到相邻的向量，从而可以用于许多具有图像和文本输入的任务，例如，Stable Diffusion的利用文字控制图片的去噪过程，实现通过文字描述左右图像的生成。

![](https://localimage.nolibox.com/article_img_url/IsF0dkJW0okpvcx0PgtckNajnZb-boxcnsWyNz5CvHlFta35gJTNH55.png)

### Frozen CLIP


Frozen CLIP 是一个已经被冻结（即不再更新）的 OpenAI CLIP 模型版本，其中 CLIP 代表 Contrastive Language-Image Pre-Training。

Frozen CLIP 是由 OpenAI 公司提供的一个固定的 CLIP 模型版本，可以通过 TensorFlow、PyTorch 等深度学习框架使用。与原始的 CLIP 模型相比，Frozen CLIP 的参数被固定在特定的数值上，因此不再需要更新。这使得 Frozen CLIP 可以用作一个图像分类器、图像检索器等等，而无需进行进一步的训练，也不能再去微调。

由于 Frozen CLIP 已经被冻结，因此它的性能可能会低于当前最新版本的 CLIP 模型。但是，Frozen CLIP 仍然可以用于许多实际应用中，特别是那些需要一个快速、轻量级的图像分类器或图像检索器的场合。

首先，您需要在 PyTorch 中导入所需的库：

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.normal import Normal
from torch.distributions import kl_divergence


可逆神经网络（Invertible Neural Networks，INNs）是一种神经网络，可以实现完全可逆的函数变换。与传统神经网络不同的是，INNs 能够在正向传递和反向传递过程中都保留输入的所有信息，从而使得反向传递过程可以完全恢复输入数据。这使得 INNs 可以用于许多有趣的应用，例如数据压缩、图像处理、密钥生成等等。

INNs 通常由一系列可逆的块组成，每个块都包含一个正向和反向传递函数。其中，反向传递函数可以计算出一个称为 Jacobian 行列式的标量，它表示每个输出值对于每个输入值的梯度。为了确保可逆性，每个块必须满足 Jacobian 行列式非零。由于每个块都是可逆的，因此整个 INN 模型也是可逆的。

INNs 可以用于许多应用，例如可逆图像生成、流形学习、密钥生成等等。在机器学习领域，INNs 通常用于概率模型的密度估计，其中模型可以同时计算出样本的概率密度和梯度，这对于训练具有梯度的生成模型非常有用。

接下来，定义一个名为 LatentDiffusion 的类，它将使用可逆神经网络（Invertible Neural Networks，INNs）实现 Latent Diffusion 技术。在这个类中，我们需要实现两个函数： __init__ 和 forward。

In [ ]:
class LatentDiffusion(nn.Module):
    def __init__(self, dim, timesteps, noise_schedule, num_flows):
        super(LatentDiffusion, self).__init__()
        self.dim = dim
        self.timesteps = timesteps
        self.noise_schedule = noise_schedule
        self.num_flows = num_flows
        self.normal = Normal(torch.tensor(0.).to(device), torch.tensor(1.).to(device))
        self.inn = INN(dim, num_flows)

    def forward(self, x, log_det, eps, reverse=False):
        if not reverse:
            z = torch.zeros_like(x)
            for t in range(self.timesteps):
                z, log_det = self.inn(z, log_det, reverse=False)
                sigma = self.noise_schedule[t]
                noise = self.normal.sample(z.shape).to(device)
                z = z + sigma * noise
            return z, log_det
        else:
            z = x
            for t in reversed(range(self.timesteps)):
                sigma = self.noise_schedule[t]
                noise = eps[t].to(device)
                z = z - sigma * noise
                z, log_det = self.inn(z, log_det, reverse=True)
            return z, log_det


In [ ]:
ds

其实我们在之前的文章《能量视角下的GAN模型（三）：生成模型=能量模型》、《从去噪自编码器到生成模型》也简单介绍过扩散模型。说到扩散模型，一般的文章都会提到能量模型（Energy-based Models）、得分匹配（Score Matching）、朗之万方程（Langevin Equation）等等，简单来说，是通过得分匹配等技术来训练能量模型，然后通过郎之万方程来执行从能量模型的采样。

从理论上来讲，这是一套很成熟的方案，原则上可以实现任何连续型对象（语音、图像等）的生成和采样。但从实践角度来看，能量函数的训练是一件很艰难的事情，尤其是数据维度比较大（比如高分辨率图像）时，很难训练出完备能量函数来；另一方面，通过朗之万方程从能量模型的采样也有很大的不确定性，得到的往往是带有噪声的采样结果。所以很长时间以来，这种传统路径的扩散模型只是在比较低分辨率的图像上做实验。

https://spaces.ac.cn/archives/9119/comment-page-1

#### 加噪过程



##### Langevin Dynamics

Langevin Dynamics（Langevin方程）是一种基于概率的动力学模型，用于描述带有阻尼和随机噪声的粒子在外力场中的运动。

Langevin Dynamics 可以用于对概率分布进行采样和优化，特别是在深度学习中，Langevin Dynamics 通常用于模型参数的优化和贝叶斯推断。在优化中，Langevin Dynamics 通过引入随机噪声，可以帮助算法跳出局部极值，从而更容易找到全局最优解。

**Langevin Dynamics 的基本形式如下**：

$$\frac{dx}{dt} = -\nabla U(x) - \gamma \frac{dx}{dt} + \sqrt{2\gamma k_BT}\frac{dW}{dt}$$

其中，$x$ 表示粒子的位置，$U(x)$ 表示势能函数，$\gamma$ 是粘滞阻力系数，$k_B$ 是玻尔兹曼常数，$T$ 是温度，$\sqrt{2\gamma k_B T}\frac{dW}{dt}$是扩散过程中的随机项即扩散项，$W(t)$ 是一个随机过程，满足 $W(t_1)-W(t_2) \sim N(0,|t_1-t_2|)$。

- 势能函数（potential energy function)：势能函数是一个物理概念，在物理学中用于描述物体在外力作用下的势能大小，是一个关于物体位置的函数。在概率论和统计学中，势能函数也被广泛应用，用于描述概率分布函数的形状和性质。在机器学习中，势能函数通常被用来描述模型的能量函数，也就是对输入数据的评估函数。对于一个给定的输入数据，势能函数将输出一个标量值，用于表示这个输入数据的“能量”大小。通常，势能函数的取值越小，表示输入数据的质量越好，这与能量在物理学中的概念是类似的。在一些概率生成模型中，如玻尔兹曼机（Boltzmann Machines）、Hopfield网络等，势能函数通常被用来描述模型的能量函数。在这些模型中，势能函数通常采用一些简单的形式，如二次型函数等，这些函数的形式足够简单，但却足以表达出模型所需的能量函数的基本特征。同时，通过对势能函数进行优化，可以得到模型的最优参数，从而使模型能够更好地拟合数据。
- 粘滞阻力系数（viscous drag coefficient）：粘滞阻力系数是一个物理概念，用于描述一个物体在流体介质中运动时受到的阻力大小。在流体动力学中，粘滞阻力系数是一个重要的参数，它与物体的形状、大小、流体介质的性质等因素有关。在机器学习中，粘滞阻力系数也被广泛应用，尤其是在Langevin Dynamics等随机过程中。在Langevin Dynamics中，一个粒子在粘性流体介质中的运动可以被描述为一个随机过程，其中粘滞阻力系数是一个关键的参数，它决定了粒子在流体介质中的受力大小。在训练可逆神经网络等模型时，Langevin Dynamics常常被用来生成高质量的样本，而粘滞阻力系数则决定了样本生成的速度和稳定性，因此在调整Langevin Dynamics的超参数时，粘滞阻力系数也是一个需要调整的关键参数之一。
- 玻尔兹曼常数（Boltzmann constant）：其通常用符号$k_B$表示，是一个物理学中的基本常数，用于描述分子热运动中的温度与能量之间的关系。具体来说，玻尔兹曼常数是描述微观粒子的温度与能量之间的比例关系的一个常数，其大小为$k_B = 1.380649 \times 10^{-23} \text{ J/K}$。其中，J代表焦耳，K代表开尔文。玻尔兹曼常数是一个基本常数，与温度的单位无关。在统计物理学和热力学中，玻尔兹曼常数通常被用于描述分子热运动的性质，如气体分子的平均动能、热容等。在机器学习中，玻尔兹曼常数也被广泛应用，尤其是在玻尔兹曼机和基于能量的模型中。在这些模型中，玻尔兹曼常数通常被用于描述系统能量的比例关系，从而可以用来计算系统的熵、自由能等物理量。
- $W(t)$ 是一个随机过程，满足 $W(t_1)-W(t_2) \sim N(0,|t_1-t_2|)$：这个随机过程 $W(t)$ 被称为布朗运动（Brownian motion），也叫做噪声过程或者渐近平稳过程。它是一个随机连续函数，通常被用来模拟微观粒子在液体或气体中的随机运动。布朗运动的最重要的特点是其增量的正态分布，即对于任意的 $t_1 > t_2$，有 $W(t_1)-W(t_2) \sim N(0, |t_1 - t_2|)$，其中 $N(0, |t_1 - t_2|)$ 表示均值为 $0$，方差为 $|t_1 - t_2|$ 的正态分布。这个特点表明布朗运动是一个自相关函数为 $0$，且具有独立增量的随机过程。在物理学、概率论、金融学和统计学等领域中，布朗运动都有着广泛的应用。在机器学习领域，布朗运动也经常被用于随机过程建模，如随机梯度下降优化算法中的噪声模型、蒙特卡罗方法中的采样过程、以及深度学习中的**噪声注入**等。

Langevin Dynamics 可以看作是一个随机微分方程，它*将随机噪声引入到粒子的运动中，使得粒子的位置会随机变化，同时也会受到阻尼的影响而趋向稳定状态。*

在实际应用中，通常采用数值方法来解决 Langevin Dynamics 方程，例如 Euler–Maruyama 方法、Milstein 方法等。这些数值方法可以通过迭代更新粒子的位置，来模拟 Langevin Dynamics 的运动过程。在机器学习中，Langevin Dynamics 通常用于对模型参数进行采样和优化，例如在贝叶斯神经网络和生成式模型中，Langevin Dynamics 可以用于对参数进行后验采样和生成样本。

##### 噪声扩散

在扩散模型中，噪声扩散是指加入随机噪声使得系统从初始状态向更高概率的状态转移的过程。在扩散过程中，噪声项$\sqrt{2\gamma k_BT}\frac{dW}{dt}$ 的扩散可以用下面的随机微分方程来描述：

$$ dX_t = -\nabla U(X_t) dt + \sigma(X_t)dW_t $$

其中，$X_t$ 是系统状态，$U$ 表示势能函数，$\nabla U(X_t)$ 是在状态 $X_t$ 下的漂移项（有的写成$\mu(X_t)$ ），$\sigma(X_t)$ 是噪声扩散系数(有的写成$\sqrt{2D}
$)，$W_t$ 是布朗运动。

> 随机微分方程（Stochastic Differential Equation，SDE）是一类描述随机现象的微分方程。它是由一个确定性的微分方程和一个随机过程组成的方程，通常用于描述物理、生物、金融等领域的随机现象。随机微分方程一般由两部分组成：漂移项和扩散项。漂移项是对应随机过程的期望漂移，扩散项则对应着随机过程的方差扩散。SDE的解也是一个随机过程，其解的概率分布可以由Fokker-Planck方程得到。常见的随机微分方程有Ornstein-Uhlenbeck过程、Langevin方程等。

> Langevin Dynamics和Langevin Equation是同一概念的不同称呼，它们都是指描述分子动力学系统中的运动规律的数学模型。在物理学和统计力学中，Langevin Dynamics通常更常用，而在数学和概率论中，Langevin Equation更常用。它们的形式是相同的，都包含了势能函数、粘滞阻力系数和随机噪声项，用于描述粒子的运动。

##### 扩散过程

噪声扩散和扩散过程在物理学和随机过程理论中是相关的概念。噪声扩散是指随机噪声对粒子或分子的运动产生的影响，而扩散过程是指在随机噪声的影响下，粒子或分子在空间中的*随机运动*。在扩散模型中，噪声扩散和扩散过程是密切相关的，噪声扩散可以被用来描述粒子或分子在扩散过程中受到的随机影响。

扩散过程是一种随机过程，用下面的随机微分方程表示：

$\mathrm{d}X_t = \mu \mathrm{d}t + \sigma \mathrm{d}W_t$
 
其中，$X_t$ 是随时间变化的随机变量，$\mu$ 是常数漂移率，$\sigma$ 是常数扩散系数，$W_t$ 是标准布朗运动（也称为 Wiener 过程）。

这个方程表示的是 $X_t$ 的微小变化，由一个漂移项 $\mu dt$ 和一个扩散项 $\sigma dW_t$ 组成。其中，$\mu dt$ 表示在无扰动情况下 $X_t$ 以常数速率 $\mu$ 变化，$\sigma dW_t$ 表示随机扰动。由于 $W_t$ 是标准布朗运动，因此 $dW_t$ 是均值为 0，方差为 $dt$ 的正态分布。

也能用偏微分方程来描述（这两个方程是等价的，它们都描述了扩散过程，只是表达方式略有不同。）：

$$\frac{\partial u(x,t)}{\partial t} = D \nabla^2 u(x,t)$$

其中 $u(x,t)$ 表示在时刻 $t$ 时位置在 $x$ 的粒子的概率密度函数，$D$ 是扩散系数，$\nabla^2$ 是拉普拉斯算子。



---


---

扩散方程可以看成是Langevin Dynamics的一个特例，是势能函数为0且粘滞阻力系数 $\gamma$ 为常数的特例时。


当势能函数为常数时，即 $U(x) = const.$ 时，Langevin Dynamics的随机微分方程变成：

$$\frac{dx}{dt} = - \gamma \frac{dx}{dt} + \sqrt{2\gamma k_BT}\frac{dW}{dt}$$
 
这是一个关于速度的一阶随机微分方程(SDE)，它描述了粒子在外部无势场作用下的随机运动。可以将其看成是扩散过程的一种特殊形式。

在 Langevin Dynamics 中，粘滞阻力系数 $\gamma$ 控制了势能函数 $U(x)$ 和噪声扩散项扩散项$\sqrt{2\gamma k_B T}\frac{dW}{dt}$之间的平衡。具体来说，$\gamma$ 越大，扩散项的影响越小，系统的行为更接近于受势能函数控制的情况；$\gamma$ 越小，扩散项的影响越大，系统的行为更接近于随机漫步的情况。因此，粘滞阻力系数可以看作是控制系统的行为在势能函数和随机噪声之间平衡的重要参数。

需要让粘滞阻力系数趋近于0。这通常出现在需要在蒙特卡洛模拟中采样分布时，因为如果粘滞阻力系数太大，则会使采样速度减慢，导致采样效率低下。此时，将粘滞阻力系数调整为接近于0的较小值可以提高采样效率，从而更快地获得更准确的采样结果。但是需要注意的是，将粘滞阻力系数调整得太小也会导致数值计算的不稳定性，因此需要在调整时进行合理的取舍。






### 去噪过程





总的来说，扩散模型通过扩散和去噪两个过程，逐步降低噪声的强度，最终实现去噪的目的。在去噪过程中，使用了随机游走、扩散算子和梯度下降等算法，结合迭代的思想，不断优化去噪效果，从而恢复出原始的图像。


之前的随机微分方程可以通过解析方法转化为偏微分方程。具体来说，我们可以将随机微分方程的解表示为一个概率密度函数，然后应用Fokker-Planck方程将概率密度函数转化为偏微分方程的形式，得到第二个方程。因此，这两个方程是等价的，可以相互转化。

解析方法：在数学中，解析方法是指通过计算或者变换来得到一个问题的解的方法，这通常是通过代数和微积分方法来实现的。在物理和工程学科中，解析方法通常用于精确地解决各种数学模型，例如微分方程、偏微分方程等。它的优点是可以提供数学上的精确解，同时可以深入理解模型背后的物理过程。不过，解析方法通常只适用于一些简单的模型，而对于复杂的模型，需要使用数值方法进行求解。

将随机微分方程的期望（或称为平均场）进行求解，就可以得到一个偏微分方程。这个偏微分方程描述的是随机微分方程的平均行为。这个方法也被称为平均场近似。

具体来说，设 $x(t)$ 是随机微分方程的解，$E[x(t)]$ 表示其期望。通过求解 $E[x(t)]$，可以得到一个偏微分方程，该方程描述了随机过程在平均意义下的行为。

举个例子，对于 Langevin Dynamics，可以通过求解期望来得到 Fokker-Planck 方程。这个方程描述了粒子在势场中的平均行为，包括扩散和漂移。同样，对于扩散过程，也可以通过平均场近似得到扩散方程，描述随机过程在平均意义下的行为。

当考虑到Langevin Dynamics中的随机性时，不再需要考虑单个粒子的运动，而需要考虑大量粒子的集体行为。可以将所有粒子的分布函数表示为 $P(x,t)$，其中 $x$ 表示粒子的位置，$t$ 表示时间。那么，粒子的集体运动可以由Fokker-Planck方程来描述：

$$\frac{\partial P(x,t)}{\partial t}=-\nabla \cdot [A(x) P(x,t)] +\frac{1}{2} \nabla \cdot [\nabla (B(x)P(x,t))]$$

其中 $A(x)$ 和 $B(x)$ 分别为漂移项和扩散项。$A(x)$ 可以通过求解 Langevin Dynamics 的平均运动来得到。假设 Langevin Dynamics 的随机项服从正态分布，则 $B(x)$ 可以通过求解 Langevin Dynamics 的均方差得到。

Fokker-Planck方程是一个偏微分方程，可以用来描述粒子的分布随时间和位置的变化。求解Fokker-Planck方程可以得到粒子分布函数在任意时间的解析解。这个解析解可以通过数值方法进行计算和模拟，从而得到粒子的集体行为。

Fokker-Planck方程：